In [1]:
import numpy as np
import tensorflow as tf


In [2]:
def conv_block(inputs,n_filters,n_conv):
    output = inputs
    for i in range(n_conv):
        output = tf.layers.conv2d(inputs=output,filters=n_filters,kernel_size=[3,3],padding='same',activation=tf.nn.relu)
    output = tf.layers.max_pooling2d(inputs=output,pool_size=[2,2],strides=2)
    return output

In [3]:
def model(features,labels,mode,params):
    x = features['x']
    for n_filters,n_conv in params['network_def']:
        x = conv_block(x,n_filters,n_conv)
    x = tf.reshape(x,[-1,params['flat_size']])
    x = tf.layers.dense(inputs=x,units=params['fc1'],activation=tf.nn.relu)
    x = tf.layers.dropout(inputs=x,rate=0.4,training=(mode==tf.estimator.ModeKeys.TRAIN))
    x = tf.layers.dense(inputs=x,units=params['fc2'],activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=x,units=params['n_classes'],activation=None)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    predictions = {
        'classes' : tf.argmax(logits,axis=1),
        'probabilities' : tf.nn.softmax(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(predictions=predictions,mode=mode)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    eval_ops = {
        'accuracy' : tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_ops)

def vgg_block(inputs,n_filters,dr,n_conv,lmd,mode):
    output = inputs
    for i in range(n_conv):
        output = tf.layers.conv2d(inputs=output,filters=n_filters,kernel_size=[3,3],padding='same',activation=tf.nn.relu,kernel_regularizer=tf.contrib.layers.l2_regularizer(lmd))
        output = tf.layers.batch_normalization(inputs=output,training=(mode==tf.estimator.ModeKeys.TRAIN))
        if i != n_conv-1:
            output = tf.layers.dropout(inputs=output,rate=dr,training=(mode==tf.estimator.ModeKeys.TRAIN))
    output = tf.layers.max_pooling2d(inputs=output,pool_size=[2,2],strides=2)
    return output

def vgg_net(features,labels,mode,params):
    x = features['x']
    for n_filters,n_conv,dr in params['network_def']:
        x = vgg_block(x,n_filters,dr,n_conv,params['regulariser'],mode)
    x = tf.layers.dropout(inputs=x,rate=0.5,training=(mode==tf.estimator.ModeKeys.TRAIN))
    x = tf.reshape(x,[-1,params['flat_size']])
    x = tf.layers.dense(inputs=x,units=params['fc1'],activation=tf.nn.relu,kernel_regularizer=tf.contrib.layers.l2_regularizer(params['regulariser']))
    x = tf.layers.batch_normalization(inputs=x,training=(mode==tf.estimator.ModeKeys.TRAIN))
    x = tf.layers.dropout(inputs=x,rate=0.5,training=(mode==tf.estimator.ModeKeys.TRAIN))
    
    logits = tf.layers.dense(inputs=x,units=params['n_classes'],activation=None)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    predictions = {
        'classes' : tf.argmax(logits,axis=1),
        'probabilities' : tf.nn.softmax(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(predictions=predictions,mode=mode)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    eval_ops = {
        'accuracy' : tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_ops)
  
    
    

In [4]:
params = {
    'network_def' : [(64,2,0.3),(128,2,0.4),(256,3,0.4),(512,3,0.4),(512,3,0.4)],
    'flat_size' : 512,
    'fc1' : 512,
    'n_classes' : 10,
    'learning_rate' : 0.01,
    'regulariser' : 0.0005
}
vgg19 = tf.estimator.Estimator(model_fn=vgg_net,params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpduzo0pv3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efe94ca7f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()
x_train = np.asarray(x_train,dtype=np.float32)
x_test = np.asarray(x_test,dtype=np.float32)
y_train = np.asarray(y_train,dtype=np.int32)
y_test = np.asarray(y_test,dtype=np.int32)
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train - mean)/(std + 1e-7)
x_test = (x_test - mean)/(std + 1e-7)

In [6]:
train_input = tf.estimator.inputs.numpy_input_fn(x={'x' : x_train},y=y_train,batch_size=128,num_epochs=None,shuffle=True)
vgg19.train(input_fn=train_input,steps=12800)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpduzo0pv3/model.ckpt.
INFO:tensorflow:loss = 3.6334805, step = 0
INFO:tensorflow:global_step/sec: 5.88256
INFO:tensorflow:loss = 2.0455441, step = 100 (17.000 sec)
INFO:tensorflow:global_step/sec: 5.97961
INFO:tensorflow:loss = 2.2386763, step = 200 (16.724 sec)
INFO:tensorflow:global_step/sec: 5.9825
INFO:tensorflow:loss = 1.9327962, step = 300 (16.715 sec)
INFO:tensorflow:global_step/sec: 5.83539
INFO:tensorflow:loss = 1.9476781, step = 400 (17.137 sec)


KeyboardInterrupt: 

In [ ]:
eval_input = tf.estimator.inputs.numpy_input_fn(x={'x' : x_test},y=y_test,num_epochs=1,shuffle=False)
eval_results = vgg19.evaluate(input_fn=eval_input)
print(eval_results)